<a href="https://colab.research.google.com/github/Geriys/VKirillV/blob/main/Seminar1%2B%D0%94%D0%971.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# Домашнее задание
# x ** 2 - 5 * x + 4 = 0
# x1 = 1
# x2 = 4

def gradient_descent(start_point, learning_rate, iterations):
    x = start_point
    for i in range(iterations):
        # Вычисляем градиент
        gradient = (2 * x - 5)*(x**2 - 5 * x + 4)
        # Обновляем значение x
        x = x - learning_rate * gradient
    return x

# # Начальная точка для x1
# start_point1 = 0
# # Начальная точка для x2
# start_point2 = 3

# Начальные точки
start_point = [0, 3]
roots = []
# Скорость обучения (learning rate)
learning_rate = 0.01

# Количество итераций
iterations = 1000

for i in start_point:
    roots.append(gradient_descent(i, learning_rate, iterations))

# Находим корни
print("Найденные корни:", roots)



Найденные корни: [0.9999999999999992, 3.9999999999999973]


In [ ]:
# задача 1

# Найти корни квадратного уравнения методом градиентного спуска
# x ** 2 - 5 * x + 4 = 0

# надо начать движение от начальной точки в направлении антградиента с заданным шагом
# x = x - lr * grad(x)
# всегда ли сойдемся за приемлемое количество шагов?
# важна ли начальная точка?
# как найти второй корень?
# как влияет ЛР?


#1 Всегда ли сойдемся за приемлемое количество шагов?
# Нет, не всегда. Метод градиентного спуска может застрять в локальных экстремумах, а не в глобальном минимуме.
# Кроме того, если функция не является гладкой, метод может потеряться в очень вытянутом или выпуклом пространстве.

#2 Важна ли начальная точка?
# Да, очень важна. Если начальная точка уже близка к корню, то метод сможет быстро сойтись.
# Если начальная точка далека, метод может застрять в локальном минимуме.

#3 Как найти второй корень?
# Для поиска второго корня, нужно начать с другой начальной точки, которая ближе к второму корню.

#4 Как влияет скорость обучения (ЛР)?
# Скорость обучения (ЛР) определяет, насколько быстро метод градиентного спуска движется в направлении антиградиента.
# Если ЛР слишком маленькое, метод может очень медленно сойтись, а если ЛР слишком большое, метод может нестабильно работать или даже расходиться.
# Выбор правильной ЛР зависит от конкретной задачи и функции, которую мы пытаемся минимизировать.

In [60]:
# Задача 2

# Реализовать адаптивний оптимизатор с подстраивающимся LR

import numpy as np

def ada_lr_optimizer(x_start, learning_rate, iterations, decay_rate=0.95, epsilon=1e-7):
    x = x_start
    cache = 0
    for i in range(iterations):
        gradient = (2 * x - 5)*(x**2 - 5 * x + 4)
        cache = decay_rate * cache + (1 - decay_rate) * gradient ** 2
        x = x - learning_rate * gradient / (np.sqrt(cache) + epsilon)
    return x

# Начальное приближение
x_start = 0
# Начальная скорость обучения
learning_rate = 0.1
# Количество итераций
iterations = 1000

root = ada_lr_optimizer(x_start, learning_rate, iterations)
print("Найденный корень:", root)


Найденный корень: 1.0000000000000002


In [62]:
import numpy as np

In [ ]:
# Task 2
# Realize forward and backward pass for linear layer with sigmoid activation

In [61]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def sigmoid_backward(da, x):
    sig = sigmoid(x)

    return da * sig * (1 - sig)

def relu(x):
    return np.maximum(0., x)

def relu_backward(da, x):
    da = np.array(da, copy = True)
    da[x <= 0] = 0
    return da

In [63]:
def mse_loss(t, y):
    return (t - y) ** 2

def d_mse_loss(t, y):
    return 2 * (y - t)


In [64]:
class LinearLayer:
    def __init__(self, n_inp, n_out, activation='sigmoid'):
        self.w = np.random.randn(n_out, n_inp) * 0.1
        self.b = np.random.randn(n_out, 1) * 0.1
        if activation == 'sigmoid':
            self.activ = sigmoid
        if activation == 'relu':
            self.activ = relu
        elif activation == 'None':
            self.activ = None
        else:
            raise Exception(f'Unknown activation "{activation}"')
        self._clear_state()

    def _clear_state(self):
        self.lin = None
        self.inp = None
        self.d_w = None
        self.d_b = None

    def forward(self, x):
        self.inp = x
        self.lin = np.dot(self.w, x) + self.b
        activ = self.activ(self.lin) if self.activ is not None else self.lin

        return activ

    def backward(self, grad): # grad = d L / d z    Dout
        # grad * dz / d lin
        if self.activ == sigmoid:
            grad_lin = sigmoid_backward(grad, self.lin)
        if self.activ == relu:
            grad_lin = relu_backward(grad, self.lin)
        else:
            grad_lin = grad
        # grad_lin * d lin / d w
        m = self.inp.shape[1]
        self.d_w = grad_lin @ self.inp.T / m
        # grad_lin * d lin / d b
        self.d_b = np.sum(grad_lin, axis=1, keepdims=True) / m

        grad = np.dot(self.w.T, grad_lin)

        return grad

In [65]:
from typing import Tuple

class Model:
    def __init__(self, arch: Tuple[Tuple[int, int]], activation):
        self.layers = []
        for i, p in enumerate(arch):
            self.layers.append(
                LinearLayer(p[0], p[1],
                            activation=activation if i < len(arch)-1 else 'None')
                )
        self._clear_state()

    def _clear_state(self):
        for l in self.layers:
            l._clear_state()

    def forward(self, x):
        for l in self.layers:
            x = l.forward(x)

        return x

    def backward(self, grad):
        for l in reversed(self.layers):
            grad = l.backward(grad)

        return grad

In [ ]:
# Task 3
# Realize SGD Momentum optimizer
# velocity = momentum * velocity - lr * gradient
# w = w + velocity

In [6]:
#для всей модели
class SGDMomentum:
    def __init__(self, model: Model, lr= 0.0001, momentum=0.9):
        self.model = model
        self.lr = lr
        self.m = momentum
        self.vel = [[np.zeros_like(layer.w),
                     np.zeros_like(layer.b)] for layer in model.layers]

    def step(self):
        for i, layer in enumerate(self.model.layers):
            self.vel[i][0] = self.vel[i][0] * self.m - self.lr * layer.d_w
            self.vel[i][1] = self.vel[i][1] * self.m - self.lr * layer.d_b
            layer.w += self.vel[i][0]
            layer.b += self.vel[i][1]

    def zero_grad(self):
        self.model._clear_state()

In [7]:
x = np.random.uniform(-3, 3, 20000)
y = x**2 + np.random.randn()*0.01


In [8]:
model = Model(((1, 100), (100, 1)), activation='relu')
optim = SGDMomentum(model, lr=0.00001)
for e in range(20):
    print(e, model.forward([[1]]), model.forward([[2]]), model.forward([[-1]]), model.forward([[-2]]))
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(np.array([[val]]))
        loss = mse_loss(t, pred)
        grad = d_mse_loss(t, pred)
        model.backward(grad)
        optim.step()



0 [[-0.29585722]] [[-0.34445304]] [[-0.19777265]] [[-0.17847653]]
1 [[1.72396487]] [[4.48079063]] [[1.59010737]] [[4.43414302]]
2 [[1.30525303]] [[4.48264826]] [[1.23017222]] [[4.45188755]]
3 [[1.11131619]] [[4.45611824]] [[1.08761939]] [[4.4366114]]
4 [[1.02328202]] [[4.41987844]] [[0.99315926]] [[4.40858837]]
5 [[1.01947311]] [[4.37907408]] [[0.91337524]] [[4.37537623]]
6 [[1.02781977]] [[4.33732487]] [[0.92380773]] [[4.33764886]]
7 [[1.02877803]] [[4.29817876]] [[0.97814804]] [[4.29962617]]
8 [[1.03592374]] [[4.26243156]] [[1.00689335]] [[4.26383623]]
9 [[1.04322506]] [[4.23040376]] [[1.02120996]] [[4.23047607]]
10 [[1.04751474]] [[4.20132799]] [[1.02810468]] [[4.19941073]]
11 [[1.04895747]] [[4.17451899]] [[1.03080319]] [[4.17037003]]
12 [[1.04846529]] [[4.14963424]] [[1.03093455]] [[4.14353788]]
13 [[1.04671801]] [[4.12659376]] [[1.02948818]] [[4.11865742]]
14 [[1.04445943]] [[4.10531487]] [[1.02716206]] [[4.09562858]]
15 [[1.04197778]] [[4.08558551]] [[1.02454423]] [[4.07422779]]

In [9]:
print(e, model.forward([[1]]), model.forward([[2]]), model.forward([[-1]]), model.forward([[103]]))

19 [[1.03103003]] [[4.01274209]] [[1.01330267]] [[475.78717601]]
